In [1]:
import requests
import json

BASE_URL = "http://127.0.0.1:8000"

def show(response):
    """Печатает статус и ответ в виде JSON с отступами."""
    print("Статус:", response.status_code)
    try:
        print("Ответ:", json.dumps(response.json(), ensure_ascii=False, indent=2))
    except Exception:
        print("Ответ:", response.text)

In [ ]:
import json
from IPython.display import display, HTML

def generate_html_from_json(json_data):
    """Генерирует полный HTML из JSON"""
    html = '''
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Объект контроля</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            h2 { color: #333; border-bottom: 2px solid #4CAF50; padding-bottom: 10px; }
            h3 { color: #555; margin-top: 25px; background-color: #f0f0f0; padding: 10px; }
            table { width: 100%; margin-bottom: 30px; border-collapse: collapse; }
            th { background-color: #4CAF50; color: white; padding: 12px; text-align: left; }
            td { padding: 10px; border: 1px solid #ddd; }
            tr:nth-child(even) { background-color: #f9f9f9; }
            .container { max-width: 1200px; margin: 0 auto; }
            .empty-section { color: #999; font-style: italic; padding: 10px; }
        </style>
    </head>
    <body>
        <div class="container">
    '''
    
    # Главный заголовок из типа
    doc_type = json_data.get('type', 'Документ')
    html += f'<h2>Тип: {doc_type}</h2>'
    
    # Обрабатываем params
    if 'params' in json_data:
        params_data = json_data['params']
        
        for section_key in sorted(params_data.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
            section_data = params_data[section_key]
            section_name = section_data.get('name', f'Раздел {section_key}')
            
            html += f'<h3>{section_key}. {section_name}</h3>'
            
            if 'params' in section_data and section_data['params']:
                section_params = section_data['params']
                
                html += '<table>'
                html += '<tr><th style="width: 40%;">Параметр</th><th>Значение</th></tr>'
                
                # Сортируем параметры по ключу
                for param_key in sorted(section_params.keys(), 
                                      key=lambda x: float(x) if '.' in str(x) else int(x) if x.isdigit() else x):
                    param_data = section_params[param_key]
                    param_name = param_data.get('name', '')
                    param_val = param_data.get('val', '')
                    
                    # Форматируем значение
                    if isinstance(param_val, list):
                        # Для длинных списков делаем переносы
                        if len(param_val) > 5:
                            val_str = '<br>'.join([str(item) for item in param_val])
                        else:
                            val_str = ', '.join([str(item) for item in param_val])
                    elif isinstance(param_val, dict):
                        # Для объектов выводим как текст
                        val_str = str(param_val)
                    else:
                        val_str = str(param_val) if param_val not in [None, "", " "] else "—"
                    
                    html += f'<tr>'
                    html += f'<td><strong>{param_name}</strong></td>'
                    html += f'<td>{val_str}</td>'
                    html += f'</tr>'
                
                html += '</table>'
            else:
                html += '<div class="empty-section">Нет данных для отображения</div>'
    
    html += '''
        </div>
    </body>
    </html>
    '''
    return html

# Альтернативная версия для прямого отображения в Jupyter
def display_json_tables_in_notebook(json_data):
    """Отображает таблицы прямо в Jupyter Notebook"""
    html_parts = []
    
    # Заголовок
    doc_type = json_data.get('type', 'Документ')
    html_parts.append(f'<h2 style="color: #2c3e50;">📋 Тип документа: {doc_type}</h2>')
    
    if 'params' in json_data:
        params_data = json_data['params']
        
        for section_key in sorted(params_data.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
            section_data = params_data[section_key]
            section_name = section_data.get('name', f'Раздел {section_key}')
            
            # Разные цвета для разных секций
            colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', 
                     '#1abc9c', '#d35400', '#34495e', '#7f8c8d', '#16a085']
            color_index = int(section_key) - 1 if section_key.isdigit() else 0
            color = colors[color_index % len(colors)]
            
            html_parts.append(f'''
            <div style="border-left: 5px solid {color}; padding-left: 15px; margin: 25px 0;">
                <h3 style="color: {color}; margin-top: 0;">
                    <span style="background-color: {color}; color: white; padding: 3px 8px; border-radius: 3px; margin-right: 10px;">
                        {section_key}
                    </span>
                    {section_name}
                </h3>
            ''')
            
            if 'params' in section_data and section_data['params']:
                section_params = section_data['params']
                
                html_parts.append('''
                <table style="width: 100%; border-collapse: collapse; margin: 10px 0;">
                    <tr style="background-color: #f8f9fa;">
                        <th style="text-align: left; padding: 12px; border: 1px solid #dee2e6; width: 40%;">
                            📝 Параметр
                        </th>
                        <th style="text-align: left; padding: 12px; border: 1px solid #dee2e6;">
                            💡 Значение
                        </th>
                    </tr>
                ''')
                
                for param_key in sorted(section_params.keys(), 
                                      key=lambda x: float(x) if '.' in str(x) else int(x) if x.isdigit() else x):
                    param_data = section_params[param_key]
                    param_name = param_data.get('name', '')
                    param_val = param_data.get('val', '')
                    
                    # Форматируем значение
                    if isinstance(param_val, list):
                        if len(param_val) > 10:  # Для очень длинных списков
                            val_str = f"<div style='max-height: 150px; overflow-y: auto;'>"
                            val_str += "<br>".join([f"• {item}" for item in param_val])
                            val_str += "</div>"
                        elif len(param_val) > 5:
                            val_str = "<br>".join([str(item) for item in param_val])
                        else:
                            val_str = ", ".join([str(item) for item in param_val])
                    elif isinstance(param_val, dict):
                        # Для объектов
                        val_str = f"<div style='font-family: monospace;'>"
                        for k, v in param_val.items():
                            val_str += f"<strong>{k}:</strong> {v}<br>"
                        val_str += "</div>"
                    else:
                        val_str = str(param_val) if param_val not in [None, "", " "] else "<em>—</em>"
                    
                    html_parts.append(f'''
                    <tr style="border-bottom: 1px solid #dee2e6;">
                        <td style="padding: 10px; border: 1px solid #dee2e6; vertical-align: top;">
                            <strong>{param_name}</strong>
                        </td>
                        <td style="padding: 10px; border: 1px solid #dee2e6; vertical-align: top;">
                            {val_str}
                        </td>
                    </tr>
                    ''')
                
                html_parts.append('</table>')
            else:
                html_parts.append('<div style="color: #95a5a6; font-style: italic; padding: 15px; background-color: #f8f9fa; border-radius: 5px;">'
                                 '⏸️ Нет данных для отображения</div>')
            
            html_parts.append('</div>')
    
    # Отображаем в Jupyter
    display(HTML(''.join(html_parts)))
    
    # Также генерируем полный HTML для сохранения
    full_html = generate_html_from_json(json_data)
    return full_html

# Загрузка вашего JSON
r4 = requests.post(f"{BASE_URL}/techcard/elementParamValue", json={"idElement": 2})
json_str= r4.json()


data = json.loads(json_str)

# Отображаем в ноутбуке
full_html = display_json_tables_in_notebook(data)

# Сохраняем в файл
with open('объект_контроля.html', 'w', encoding='utf-8') as f:
    f.write(full_html)

print("✅ HTML файл сохранен как 'объект_контроля.html'")

📝 Параметр,💡 Значение
Объект контроля,id: 2name: Труба d273x8+d108x5; АТС-0001-21
Контролируемый элемент,"Сварное соединение, Стыковое сварное соединение, Кольцевое сварное соединение"
Категория по ПНАЭ Г-7-010-89,IIIIIIIIаIIаIIIaIIIвIIIc
Размеры контролируемых элементов,—
"Наружный диаметр, D, мм",108.0
Тип сварного соединения,• C1• C2• C3• C4• C5• C6• C7• C8• C9• C10• C11• C12• C13• C14• C15• C16• C17• C18• C19• C20• C21• C22• C23• C24• C25• C26• C27• C28• C29• C30• C31• C32• C33• C34• C35• C36• C37• C38• C39• C40• C41• C42• C43• C44• C45• У1• У2• У3• У4• У5• У6• У7• У8• У9• У10• У11• Т1• Т2• Т3• Т4• Т5• Т6• Т7• Т8• Т9• Т10• Т11• Т12• Н1• Н2


✅ HTML файл сохранен как 'объект_контроля.html'


In [13]:
import json
from IPython.display import display, HTML

def generate_html_from_json(json_data):
    """Генерирует полный HTML из JSON"""
    html = '''
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Объект контроля</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            h2 { color: #333; border-bottom: 2px solid #4CAF50; padding-bottom: 10px; }
            h3 { color: #555; margin-top: 25px; background-color: #f0f0f0; padding: 10px; }
            table { width: 100%; margin-bottom: 30px; border-collapse: collapse; }
            th { background-color: #4CAF50; color: white; padding: 12px; text-align: left; }
            td { padding: 10px; border: 1px solid #ddd; }
            tr:nth-child(even) { background-color: #f9f9f9; }
            .container { max-width: 1200px; margin: 0 auto; }
            .empty-section { color: #999; font-style: italic; padding: 10px; }
            .param-number { color: #666; font-size: 0.9em; margin-right: 5px; }
        </style>
    </head>
    <body>
        <div class="container">
    '''
    
    # Главный заголовок из типа
    doc_type = json_data.get('type', 'Документ')
    html += f'<h2>Тип: {doc_type}</h2>'
    
    # Обрабатываем params
    if 'params' in json_data:
        params_data = json_data['params']
        
        for section_key in sorted(params_data.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
            section_data = params_data[section_key]
            section_name = section_data.get('name', f'Раздел {section_key}')
            
            html += f'<h3>{section_key}. {section_name}</h3>'
            
            if 'params' in section_data and section_data['params']:
                section_params = section_data['params']
                
                html += '<table>'
                html += '<tr><th style="width: 40%;">Параметр</th><th>Значение</th></tr>'
                
                # Сортируем параметры по ключу
                for param_key in sorted(section_params.keys(), 
                                      key=lambda x: float(x) if '.' in str(x) else int(x) if x.isdigit() else x):
                    param_data = section_params[param_key]
                    param_name = param_data.get('name', '')
                    param_val = param_data.get('val', '')
                    
                    # Создаем номер параметра в формате "раздел.параметр"
                    if param_key.replace('.', '').isdigit():
                        param_number = f"{section_key}.{param_key}"
                    else:
                        param_number = f"{section_key}.{param_key}"
                    
                    # Форматируем значение
                    if isinstance(param_val, list):
                        # Для длинных списков делаем переносы
                        if len(param_val) > 5:
                            val_str = '<br>'.join([f"• {str(item)}" for item in param_val])
                        else:
                            val_str = ', '.join([str(item) for item in param_val])
                    elif isinstance(param_val, dict):
                        # Для объектов выводим как текст
                        val_str = str(param_val)
                    else:
                        val_str = str(param_val) if param_val not in [None, "", " "] else "—"
                    
                    html += f'<tr>'
                    html += f'<td><span class="param-number">{param_number}</span><strong>{param_name}</strong></td>'
                    html += f'<td>{val_str}</td>'
                    html += f'</tr>'
                
                html += '</table>'
            else:
                html += '<div class="empty-section">Нет данных для отображения</div>'
    
    html += '''
        </div>
    </body>
    </html>
    '''
    return html

def display_json_tables_in_notebook(json_data):
    """Отображает таблицы прямо в Jupyter Notebook с нумерацией параметров"""
    html_parts = []
    
    # Заголовок
    doc_type = json_data.get('type', 'Документ')
    html_parts.append(f'<h2 style="color: #2c3e50; border-bottom: 2px solid #4CAF50; padding-bottom: 10px;">📋 Тип документа: {doc_type}</h2>')
    
    if 'params' in json_data:
        params_data = json_data['params']
        
        for section_key in sorted(params_data.keys(), key=lambda x: int(x) if x.isdigit() else float('inf')):
            section_data = params_data[section_key]
            section_name = section_data.get('name', f'Раздел {section_key}')
            
            # Разные цвета для разных секций
            colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', 
                     '#1abc9c', '#d35400', '#34495e', '#7f8c8d', '#16a085']
            color_index = int(section_key) - 1 if section_key.isdigit() else 0
            color = colors[color_index % len(colors)]
            
            html_parts.append(f'''
            <div style="border-left: 5px solid {color}; padding-left: 15px; margin: 30px 0 20px 0; background-color: #fafafa; border-radius: 5px; padding: 15px;">
                <h3 style="color: {color}; margin: 0 0 15px 0; font-size: 1.3em;">
                    <span style="background-color: {color}; color: white; padding: 5px 12px; border-radius: 4px; margin-right: 12px; font-weight: bold;">
                        {section_key}
                    </span>
                    {section_name}
                </h3>
            ''')
            
            if 'params' in section_data and section_data['params']:
                section_params = section_data['params']
                
                html_parts.append('''
                <table style="width: 100%; border-collapse: collapse; margin: 15px 0 25px 0; box-shadow: 0 2px 5px rgba(0,0,0,0.1);">
                    <tr style="background-color: #f8f9fa;">
                        <th style="text-align: left; padding: 14px; border: 1px solid #dee2e6; width: 45%; font-size: 1.1em;">
                            📝 Параметр
                        </th>
                        <th style="text-align: left; padding: 14px; border: 1px solid #dee2e6; font-size: 1.1em;">
                            💡 Значение
                        </th>
                    </tr>
                ''')
                
                for param_key in sorted(section_params.keys(), 
                                      key=lambda x: float(x) if '.' in str(x) else int(x) if x.isdigit() else x):
                    param_data = section_params[param_key]
                    param_name = param_data.get('name', '')
                    param_val = param_data.get('val', '')
                    
                    # Создаем номер параметра
                    if param_key.replace('.', '').isdigit():
                        param_number = f"{section_key}.{param_key}"
                    else:
                        param_number = f"{section_key}.{param_key}"
                    
                    # Форматируем значение
                    if isinstance(param_val, list):
                        if len(param_val) > 15:  # Для очень длинных списков
                            val_str = f'''
                            <div style='max-height: 200px; overflow-y: auto; padding: 5px; background-color: #f8f9fa; border-radius: 4px;'>
                                <div style="font-weight: bold; margin-bottom: 5px; color: #666;">Всего элементов: {len(param_val)}</div>
                                <div style="column-count: 2; column-gap: 20px;">
                            '''
                            for item in param_val:
                                val_str += f"<div style='margin-bottom: 3px;'>• {item}</div>"
                            val_str += "</div></div>"
                        elif len(param_val) > 8:
                            val_str = "<div style='max-height: 120px; overflow-y: auto;'>"
                            val_str += "<br>".join([f"• {item}" for item in param_val])
                            val_str += "</div>"
                        elif len(param_val) > 3:
                            val_str = "<br>".join([f"• {item}" for item in param_val])
                        else:
                            val_str = ", ".join([str(item) for item in param_val])
                    elif isinstance(param_val, dict):
                        # Для объектов
                        val_str = f"<div style='font-family: monospace; background-color: #f8f9fa; padding: 8px; border-radius: 4px;'>"
                        for k, v in param_val.items():
                            val_str += f"<strong>{k}:</strong> {v}<br>"
                        val_str += "</div>"
                    else:
                        val_str = str(param_val) if param_val not in [None, "", " "] else "<em style='color: #999;'>— не указано —</em>"
                    
                    html_parts.append(f'''
                    <tr style="border-bottom: 1px solid #eaeaea;">
                        <td style="padding: 12px; border: 1px solid #dee2e6; vertical-align: top; background-color: white;">
                            <div style="margin-bottom: 5px;">
                                <span style="color: {color}; font-weight: bold; background-color: #f0f7ff; padding: 2px 8px; border-radius: 3px; font-size: 0.9em;">
                                    {param_number}
                                </span>
                            </div>
                            <strong style="font-size: 1.05em;">{param_name}</strong>
                        </td>
                        <td style="padding: 12px; border: 1px solid #dee2e6; vertical-align: top; background-color: white;">
                            {val_str}
                        </td>
                    </tr>
                    ''')
                
                html_parts.append('</table>')
            else:
                html_parts.append('''
                <div style="color: #95a5a6; font-style: italic; padding: 20px; 
                            background-color: #f8f9fa; border-radius: 5px; text-align: center; margin: 20px 0;">
                    ⏸️ Нет данных для отображения
                </div>
                ''')
            
            html_parts.append('</div>')
    
    # Отображаем в Jupyter
    display(HTML(''.join(html_parts)))
    
    # Также генерируем полный HTML для сохранения
    full_html = generate_html_from_json(json_data)
    return full_html

# Пример использования с вашим JSON
import requests

# Ваш код для получения данных
# r4 = requests.post(f"{BASE_URL}/techcard/elementParamValue", json={"idElement": 2})
# data = r4.json()

# Для теста используем ваш пример JSON
r4 = requests.post(f"{BASE_URL}/techcard/elementParamValue", json={"idElement": 3})
json_str= r4.json()


data = json.loads(json_str)

# Отображаем в ноутбуке с нумерацией параметров
full_html = display_json_tables_in_notebook(data)

# Сохраняем в файл
with open('объект_контроля.html', 'w', encoding='utf-8') as f:
    f.write(full_html)

print("✅ HTML файл сохранен как 'объект_контроля.html'")

# Создаем ссылку для скачивания
from IPython.display import FileLink
display(FileLink('объект_контроля.html', result_html_prefix="⬇️ Скачать HTML файл: "))

📝 Параметр,💡 Значение
1.0 Объект контроля,id: 3name: труба d57x3; АТС-0001-21
1.2 Контролируемый элемент,"Сварное соединение, Стыковое сварное соединение, Кольцевое сварное соединение"
1.3 Категория по ПНАЭ Г-7-010-89,• I• II• III• IIа• IIа• IIIa• IIIв• IIIc
1.4 Размеры контролируемых элементов,— не указано —
"1.4.1 Наружный диаметр, D, мм",57.0
"1.4.2 Толщина, S, мм",3.0
1.5 Тип сварного соединения,Всего элементов: 70 • C1• C2• C3• C4• C5• C6• C7• C8• C9• C10• C11• C12• C13• C14• C15• C16• C17• C18• C19• C20• C21• C22• C23• C24• C25• C26• C27• C28• C29• C30• C31• C32• C33• C34• C35• C36• C37• C38• C39• C40• C41• C42• C43• C44• C45• У1• У2• У3• У4• У5• У6• У7• У8• У9• У10• У11• Т1• Т2• Т3• Т4• Т5• Т6• Т7• Т8• Т9• Т10• Т11• Т12• Н1• Н2


✅ HTML файл сохранен как 'объект_контроля.html'


e:\sem7\diplom\beckTechCard\beck\controllers\Test\объект_контроля.html